In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf

import cv2
import os
import shutil

from glob import glob
from pathlib import Path

In [2]:
!pip install wolta

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.0/899.0 kB 17.4 MB/s eta 0:00:00


In [4]:
os.environ['WANDB_MODE'] = 'disabled'

# Data Analysis

In [5]:
for dirname, _, _ in os.walk('/kaggle/input/cifake-real-and-ai-generated-synthetic-images'):
    print(dirname)

/kaggle/input/cifake-real-and-ai-generated-synthetic-images
/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test
/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE
/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL
/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train
/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE
/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL


In [6]:
p_paths = glob('/kaggle/input/cifake-real-and-ai-generated-synthetic-images/*')
d_paths = []

for p_path in p_paths:
    d_paths.extend(glob('{}/*'.format(p_path)))
print(d_paths)

['/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/FAKE', '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test/REAL', '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/FAKE', '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train/REAL']


In [7]:
i_paths = []

for d_path in d_paths:
    i_paths.extend(glob('{}/*'.format(d_path)))

print(len(i_paths))

120000


In [8]:
from wolta.visual_tools import get_extensions

get_extensions(i_paths)

{'jpg': 120000}

In [9]:
from wolta.visual_tools import dataset_size_same

dataset_size_same(i_paths)

True

In [10]:
temp_img = cv2.imread(i_paths[0])
ratio = temp_img.shape[1] / temp_img.shape[0]

print('Width: {}'.format(temp_img.shape[1]))
print('Height: {}'.format(temp_img.shape[0]))
print('Ratio: {}'.format(ratio))
print(temp_img.shape)

Width: 32
Height: 32
Ratio: 1.0
(32, 32, 3)


# Image Stacking

In [11]:
os.makedirs('/kaggle/working/raw')

In [12]:
for d_path in d_paths:
    current_dir = Path(d_path).name
    current_path = '/kaggle/working/raw/{}'.format(current_dir) 
    os.makedirs(current_path, exist_ok=True)

    i_paths = glob('{}/*'.format(d_path))

    for i_path in i_paths:
        shutil.copy(i_path, current_path)

# Image Splitting

In [13]:
from wolta.visual_tools import dir_split

dir_split('/kaggle/working/raw', '/kaggle/working/data', test_size=0.2, val_size=0.2)

In [14]:
# shutil.rmtree('/kaggle/working/raw')

In [15]:
# shutil.copytree('/kaggle/input/cifake-data-processed/data', '/kaggle/working/data')

In [16]:

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import random
from PIL import Image, ImageEnhance, ImageOps

def resize_image(image, target_size):
    """Randomly resize the image and then resize it back to the original size."""
    resize_factor = random.uniform(0.8, 1.2)  # Resize between 80% to 120%
    new_size = (int(image.shape[1] * resize_factor), int(image.shape[0] * resize_factor))
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    pil_img = pil_img.resize(new_size, Image.Resampling.LANCZOS)  # Resize
    pil_img = pil_img.resize(target_size, Image.Resampling.LANCZOS)  # Resize back to original size
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def rotate_image(image, target_size):
    """Randomly rotate the image and crop it to the original size."""
    angle = random.uniform(-90, 90)  # Rotate between -30 to 30 degrees
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    pil_img = pil_img.rotate(angle, expand=True, fillcolor=(255, 255, 255))

    # Center crop back to the original size
    pil_img = ImageOps.fit(pil_img, target_size, Image.Resampling.LANCZOS)
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def adjust_sharpness(image):
    """Randomly adjust sharpness of the image."""
    sharpness_factor = random.uniform(0.5, 2.0)  # Sharpness between 0.5x to 2x
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    enhancer = ImageEnhance.Sharpness(pil_img)
    pil_img = enhancer.enhance(sharpness_factor)
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def zoom_image(image, target_size):
    """Randomly zoom the image and resize it back to the original size."""
    zoom_factor = random.uniform(0.8, 1.2)  # Zoom between 80% to 120%
    zoom_size = (int(image.shape[1] * zoom_factor), int(image.shape[0] * zoom_factor))
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    pil_img = pil_img.resize(zoom_size, Image.Resampling.LANCZOS)

    # Center crop back to the original size
    pil_img = ImageOps.fit(pil_img, target_size, Image.Resampling.LANCZOS)
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def horizontal_flip_image(image):
    """Randomly flip the image horizontally."""
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    pil_img = ImageOps.mirror(pil_img)  # Flip horizontally
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

def augment_image_randomly(image):
    """
    Randomly select and apply one augmentation technique to the image.
    """
    target_size = (image.shape[1], image.shape[0])  # Original width and height
    augmentations = [
        lambda img: resize_image(img, target_size),
        lambda img: rotate_image(img, target_size),
        adjust_sharpness,
        lambda img: zoom_image(img, target_size),
        horizontal_flip_image
    ]
    augmentation = random.choice(augmentations)
    return augmentation(image)





n_dropped_patches = 0
def add_gaussian_noise(image, mean=0, std=1):
    """Add Gaussian noise to an image."""
    noise = np.random.normal(mean, std, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image


def reduce_resolution(image, target_size=(16, 16)):
    """Reduce and then restore the resolution of an image."""
    low_res = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
    restored = cv2.resize(low_res, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
    return restored

def extract_patches(image, patch_size=1):
    """
    Extract patches from an image.
    Returns a list of flattened patch intensities.
    """
    height, width, _ = image.shape
    patches = []
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = image[i:i+patch_size, j:j+patch_size].flatten()
            patches.append(np.mean(patch))  # Use mean intensity as a feature
    return patches

def compute_global_correlation(data_dir, categories, patch_size=1):
    """
    Compute the correlation of patches across the entire dataset with the label.
    """
    all_features = []
    all_labels = []

    for category, label in categories.items():
        category_path = os.path.join(data_dir, category)
        for img_name in tqdm(os.listdir(category_path)):
            img_path = os.path.join(category_path, img_name)
            if os.path.isfile(img_path):
                image = cv2.imread(img_path)
                if image is not None:
                    # print("image shape", image.shape)
                    features = extract_patches(image, patch_size)
                    # print("total patches:", len(features))
                    all_features.append(features)
                    all_labels.append(label)
    # print(len(all_features))
    all_features = StandardScaler().fit_transform(all_features)
    # Convert to a DataFrame for correlation calculation
    feature_matrix = pd.DataFrame(all_features)  # Rows = images, Columns = patches
    print("Feature_Matrix.Head()..........")
    print(feature_matrix.head())
    label_series = pd.Series(all_labels, name="label")
    # print(label_series)
    correlations = feature_matrix.corrwith(label_series)  # Correlation of each patch with the label
    print("Correlation.Head()...........")
    print(correlations.head())
    print(f"Max correlation: {abs(correlations.max())}")
    print(f"Min correlation: {abs(correlations.min())}")
    print(f"Max correlation index: {abs(correlations).idxmax()}")
    print(f"Size of correlation: {correlations.size}")

    return correlations

def drop_least_correlated_features(image, correlations, patch_size=1, threshold=0.05):
    """
    Drop patches with the least correlation to the label.
    """
    height, width, _ = image.shape
    dropped_image = image.copy()
    patch_idx = 0

    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            if patch_idx < len(correlations) and abs(correlations[patch_idx]) < threshold:
                dropped_image[i:i+patch_size, j:j+patch_size] = 0  # Drop patch (set to black)
                # n_dropped_patches += 1
            patch_idx += 1

    return dropped_image

def process_dataset_with_correlation(data_dir, output_dir, correlations, patch_size=1, threshold=0.05):
    """
    Process the entire dataset by dropping least correlated patches and saving.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)


    for img_name in tqdm(os.listdir(data_dir)):
        img_path = os.path.join(data_dir, img_name)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is not None:
                processed_image = drop_least_correlated_features(image, correlations, patch_size, threshold)
                output_img_path = os.path.join(output_dir, img_name)
                cv2.imwrite(output_img_path, processed_image)
                    


# def normalize_image(image):
#     """Normalize image to range [0, 1]."""
#     return image / 255.0
    
def process_and_save_images(input_path, output_path, preprocess_fn, augment = True):
    """Process images with a given preprocessing function and save."""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    image_paths = os.listdir(input_path)
    for img_name in tqdm(image_paths):
        img_path = os.path.join(input_path, img_name)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is None:
                continue
            processed_image = preprocess_fn(image)
            if augment:
                img_name = img_name.split('.')[0] + '_1.' + img_name.split('.')[1]
            output_img_path = os.path.join(output_path, img_name)
            cv2.imwrite(output_img_path, (processed_image * 255).astype(np.uint8) if processed_image.max() <= 1 else processed_image)


base_dir = "/kaggle/working/data"  # Replace with the root path of your dataset
subsets = [ "train","val"]
categories = {"REAL": 1, "FAKE": 0}
train_data_dir = "/kaggle/working/data/train"  # Training data directory

test_real_path = "/kaggle/working/data/test/REAL"
test_fake_path = "/kaggle/working/data/test/FAKE"

print(f"Processing test/REAL...")
process_and_save_images(test_real_path, test_real_path, lambda img: add_gaussian_noise(img))
print(f"Processing test/FAKE...")
process_and_save_images(test_fake_path, test_fake_path, lambda img: add_gaussian_noise(img))

# Apply the Robustness Methods to train and val set
for subset in subsets:
    for category, label in categories.items():
        input_path = os.path.join(base_dir, subset, category)
        # output_path = os.path.join(base_dir, f"{subset}_{category}_processed")
        
        print(f"Processing {subset}/{category} for random augmentation...")
        process_and_save_images(input_path, input_path, lambda img: augment_image_randomly(img))


print("Computing global correlation...")
correlations = compute_global_correlation(train_data_dir, categories, patch_size=1)

all_subsets = [ "train","val", "test"]

# drop least correlated features from all sets 
for subset in all_subsets:
    for category, label in categories.items():
        # n_dropped_patches = 0
        input_path = os.path.join(base_dir, subset, category)        
        print(f"Processing {subset}/{category} for feature dropping...")
        process_dataset_with_correlation(input_path, input_path, correlations, patch_size=1, threshold=0.01)

# Apply the Robustness Methods to train and val set
# for subset in all_subsets:
#     for category, label in categories.items():
#         input_path = os.path.join(base_dir, subset, category)
#         # output_path = os.path.join(base_dir, f"{subset}_{category}_processed")
        
#         # print(f"Processing {subset}/{category} for reduced resolution...")
#         # process_and_save_images(input_path, input_path, lambda img: reduce_resolution(img))
        
        
#         print(f"Processing {subset}/{category} for normalization...")
#         process_and_save_images(input_path, input_path, lambda img: normalize_image(img), augment = False)



Processing test/REAL...


100%|██████████| 10000/10000 [00:02<00:00, 3694.55it/s]


Processing test/FAKE...


100%|██████████| 12000/12000 [00:03<00:00, 3596.94it/s]


Processing train/REAL for random augmentation...


...

100%|██████████| 24000/24000 [01:18<00:00, 306.16it/s]


Processing test/REAL for feature dropping...


100%|██████████| 20000/20000 [01:05<00:00, 305.27it/s]


Processing test/FAKE for feature dropping...


100%|██████████| 24000/24000 [01:18<00:00, 305.85it/s]


In [17]:
# shutil.rmtree('/kaggle/working/kaggle')
# shutil.copytree('/kaggle/input/cifake-data-processed/data', '/kaggle/working/data_original')

In [18]:
# original_data_dir = "/kaggle/working/data_original"
# data_dir = "/kaggle/working/data"
# for subset in all_subsets:
#     for category, label in categories.items():
#         input_path = os.path.join(original_data_dir, subset, category)
#         output_path = os.path.join(data_dir, subset, category)
#         if not os.path.exists(output_path):
#             os.makedirs(output_path)
#         image_paths = os.listdir(input_path)
#         if len(image_paths) > 0.1 * len(image_paths):
#             image_paths = image_paths[int(0.1 * len(image_paths)):]
#             for img_name in image_paths:
#                 img_path = os.path.join(input_path, img_name)
#                 shutil.copy(img_path, output_path)

In [19]:
# # Drop 80% of the images in the training set
# all_subsets = [ "train","val", "test"]

# for subset in all_subsets:
#     for category, label in categories.items():
#         input_path = os.path.join(base_dir, subset, category)
#         image_paths = os.listdir(input_path)
#         if len(image_paths) > 0.2 * len(image_paths):
#             image_paths = image_paths[:int(0.8 * len(image_paths))]
#             for img_name in image_paths:
#                 img_path = os.path.join(input_path, img_name)
#                 os.remove(img_path)

In [20]:
# # Apply the Robustness Methods to train and val set
# for subset in subsets:
#     for category, label in categories.items():
#         input_path = os.path.join(base_dir, subset, category)
#         # output_path = os.path.join(base_dir, f"{subset}_{category}_processed")
        
#         print(f"Processing {subset}/{category} for random augmentation...")
#         process_and_save_images(input_path, input_path, lambda img: augment_image_randomly(img))

In [21]:
# n_dropped_patches = 0
# def drop_least_correlated_features(image, correlations, patch_size=1, threshold=0.1):
#     """
#     Drop patches with the least correlation to the label.
#     """
#     height, width, _ = image.shape
#     dropped_image = image.copy()
#     patch_idx = 0
#     n_dropped_patches = 0
#     for i in range(0, height, patch_size):
#         for j in range(0, width, patch_size):
#             if patch_idx < len(correlations) and abs(correlations[patch_idx]) < threshold:
#                 dropped_image[i:i+patch_size, j:j+patch_size] = 0  # Drop patch (set to black)
#                 n_dropped_patches += 1
#             patch_idx += 1
#     print(f"Dropped {n_dropped_patches} patches in this image")
#     return dropped_image, n_dropped_patches
# def process_dataset_with_correlation(data_dir, output_dir, correlations, patch_size=1, threshold=0.1):
#     """
#     Process the entire dataset by dropping least correlated patches and saving.
#     """
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     n_dropped_patches = 0
#     for img_name in tqdm(os.listdir(data_dir)):
#         img_path = os.path.join(data_dir, img_name)
#         if os.path.isfile(img_path):
#             image = cv2.imread(img_path)
#             if image is not None:
#                 processed_image, n_d = drop_least_correlated_features(image, correlations, patch_size, threshold)
#                 n_dropped_patches += n_d
#                 output_img_path = os.path.join(output_dir, img_name)
#                 cv2.imwrite(output_img_path, processed_image)
#     return n_dropped_patches 
                
# all_subsets = [ "train","val", "test"]

# # drop least correlated features from all sets 
# for subset in all_subsets:
#     for category, label in categories.items():
#         n_dropped_patches = 0
#         input_path = os.path.join(base_dir, subset, category)   
#         output_path = os.path.join(base_dir, subset, category+"_processed")
#         print(f"Processing {subset}/{category} for feature dropping...")
#         n_dropped_patches = process_dataset_with_correlation(input_path, input_path, correlations, patch_size=1, threshold=0.02)
#         print(f"Dropped {n_dropped_patches} patches in total in images of {subset}/{category}.")

In [22]:
# !rm -r '/kaggle/working/data/train.cache'
# !rm -r '/kaggle/working/data/val.cache'
# !rm -r '/kaggle/working/data/test.cache'
# !rm -r '/kaggle/working/runs'



In [23]:
# for subset in subsets:
#     for category, label in categories.items():
#         input_path = os.path.join(base_dir, subset, category)
#         # output_path = os.path.join(base_dir, f"{subset}_{category}_processed")
        
#         print(f"Processing {subset}/{category} for reduced resolution...")
#         process_and_save_images(input_path, input_path, lambda img: reduce_resolution(img))

In [24]:
from wolta.visual_tools import cls_img_counter

cls_img_counter('/kaggle/working/data')

{'REAL': 100000, 'FAKE': 120000}

# YOLO

In [25]:
from ultralytics import YOLO

model = YOLO(model='yolo11x-cls.pt')
results = model.train(data='/kaggle/working/data', epochs=5, imgsz=32, verbose= True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 56.9M/56.9M [00:00<00:00, 192MB/s]


Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolo11x-cls.pt, data=/kaggle/working/data, epochs=5, time=None, patience=100, batch=16, imgsz=32, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 5.35M/5.35M [00:00<00:00, 74.0MB/s]


AMP: checks passed ✅


...

               classes   top1_acc   top5_acc: 100%|██████████| 1375/1375 [00:28<00:00, 48.33it/s]

                   all      0.911          1



5 epochs completed in 0.792 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 57.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 57.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 132000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 44000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 1375/1375 [00:21<00:00, 64.47it/s]


                   all      0.911          1
Speed: 0.0ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [26]:
test_results = model.val(data='/kaggle/working/data', imgsz=32, split="test")

Ultralytics 8.3.50 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLO11x-cls summary (fused): 227 layers, 28,334,978 parameters, 0 gradients, 110.3 GFLOPs
train: /kaggle/working/data/train... found 132000 images in 2 classes ✅ 
val: /kaggle/working/data/val... found 44000 images in 2 classes ✅ 
test: /kaggle/working/data/test... found 44000 images in 2 classes ✅ 


test: Scanning /kaggle/working/data/test... 44000 images, 0 corrupt: 100%|██████████| 44000/44000 [00:20<00:00, 2146.51it/s]


test: New cache created: /kaggle/working/data/test.cache


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
               classes   top1_acc   top5_acc: 100%|██████████| 2750/2750 [00:33<00:00, 81.58it/s]


                   all      0.736          1
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [27]:
# # Save the trained model
model.save('trained_yolo_model.pt')
shutil.rmtree('/kaggle/working/data')
# shutil.rmtree('/kaggle/working/raw')
